# SSIM
https://medium.com/@danielyogatama.dy/ssim-on-python-eb1a76a2799b

In [1]:
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim

In [ ]:

# Load images
image1 = cv2.imread(image1)
image2 = cv2.imread(image2)
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
# SSIM Score: 0.38

In [ ]:
def ssim_compare(img1_path, img2_path) :
    img1 = cv2.imread(img1_path, 0)
    img2 = cv2.imread(img2_path, 0)
    dim = (6022,5513)
    # print("Img1 Resolution:", img1.shape)
    # print("Img2 Resolution:", img2.shape)
    img1 = cv2.resize(img1, dim)
    img2 = cv2.resize(img2, dim)
    # print("Img1 Res :", img1.shape)
    # print("Img2 Res :", img2.shape)
    ssim_score, dif = ssim(img1, img2, full=True)
    return ssim_score


In [4]:
import os
import re
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim

# 1) Helper to compute SSIM on two RGB images
def compute_ssim(path1, path2):
    # load as H×W×3 uint8 arrays
    im1 = np.array(Image.open(path1).convert("RGB"))
    im2 = np.array(Image.open(path2).convert("RGB"))
    # multichannel=True lets SSIM run on color
    score, _ = ssim(im1, im2, multichannel=True, full=True, data_range=im1.max()-im1.min())
    return score

def stats_ssim_from_folder(folder):

    pattern = re.compile(r"(.+)_cs\.(png|jpe?g|bmp)$", re.IGNORECASE)
    entries = []  # will hold (ssim_score, base, stylized_path)

    for fname in sorted(os.listdir(folder)):
        m = pattern.match(fname)
        if not m:
            continue
        base, ext = m.groups()
        stylized_path = os.path.join(folder, fname)
        content_path  = os.path.join(folder, f"{base}_c.{ext}")

        if not os.path.isfile(content_path):
            print(f"  → skipping {base}: missing content file")
            continue

        try:
            score = compute_ssim(stylized_path, content_path)
        except Exception as e:
            print(f"  ⚠️ failed on {base}: {e}")
            continue

        entries.append((score, base, stylized_path))

    if not entries:
        print("No valid *_cs / *_c pairs found.")
        return

    # aggregate
    scores = [e[0] for e in entries]
    avg   = sum(scores) / len(scores)
    mn_e  = min(entries, key=lambda x: x[0])
    mx_e  = max(entries, key=lambda x: x[0])

    print(f"Computed SSIM on {len(entries)} pairs:")
    print(f"  Average SSIM = {avg:.4f}")
    print(f"  Min  SSIM = {mn_e[0]:.4f}  ← {mn_e[2]}")
    print(f"  Max  SSIM = {mx_e[0]:.4f}  ← {mx_e[2]}")

if __name__ == "__main__":
    import argparse
    p = argparse.ArgumentParser()
    p.add_argument("folder", help="../AdaAttN/results/AdaAttN_test/test_latest/images")
    args = p.parse_args()

    stats_ssim_from_folder(args.folder)


usage: ipykernel_launcher.py [-h] folder
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/opt/anaconda-2024.02/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import os
import re
import numpy as np
from PIL import Image
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from pathlib import Path
import shutil

def compute_ssim(path1, path2):
    # load as H×W×3 uint8 arrays
    img1 = cv2.imread(path1)
    img2 = cv2.imread(path2)

    dim = (512, 512)
    img1 = cv2.resize(img1, dim)
    img2 = cv2.resize(img2, dim)

    # multichannel=True lets SSIM run on color
    score = ssim(img1, img2, channel_axis=2, data_range=img2.max()-img2.min()) # Running TEST
    return score

def stats_ssim_from_folder(folder, outname, model="AdaAttN"):
    content_dir = "../data/content_test_small"
    #content_dir = "../StyTR-2/input/content"
    pattern = re.compile(r"(.+)_cs\.(png|jpg|jpeg|bmp)$", re.IGNORECASE)
    out = []
    
    for fname in sorted(os.listdir(folder)):
        if model == "AdaAttN":
            m = pattern.match(fname)
            if not m:
                continue
                
            stylized_path = os.path.join(folder, fname)
            content_path = stylized_path.replace("_cs.", "_c.")
            
        elif model=="StyTr2":
            if not fname.lower().endswith(".jpg"):
                continue  # Skip non-JPG files
                
            stylized_path = os.path.join(folder, fname)
            content_path = os.path.join(content_dir, 
                                        re.match(r'^(.*?)_stylized_', fname).group(1) + ".jpg")
            

        if not os.path.isfile(content_path):
            print(f"  → skipping {fname}: missing content file")
            continue

        try:
            score = compute_ssim(stylized_path, content_path)
        except Exception as e:
            print(f"  ⚠️ failed on {fname}: {e}")
            continue

        out.append(score)
    
    with open("SSIM_" + outname + ".txt", "w") as f:
        f.write(str(np.mean(out)) + "\n")

In [8]:
stats_ssim_from_folder("../AdaAttN/results/AdaAttN_test/test_latest/images", "test" , model="AdaAttN")

In [9]:
stats_ssim_from_folder("../StyTR-2/out", "test", model="StyTr2")

  → skipping 14_stylized_20200428220829.jpg: missing content file
  → skipping 14_stylized_3314.jpg: missing content file
  → skipping 14_stylized_feathers.jpg: missing content file
  → skipping 14_stylized_rain_princess.jpg: missing content file
  → skipping 14_stylized_udnie.jpg: missing content file
  → skipping 14_stylized_wave.jpg: missing content file
  → skipping RoundBales_stylized_20200428220829.jpg: missing content file
  → skipping RoundBales_stylized_3314.jpg: missing content file
  → skipping RoundBales_stylized_feathers.jpg: missing content file
  → skipping RoundBales_stylized_rain_princess.jpg: missing content file
  → skipping RoundBales_stylized_udnie.jpg: missing content file
  → skipping RoundBales_stylized_wave.jpg: missing content file
  → skipping bair_stylized_20200428220829.jpg: missing content file
  → skipping bair_stylized_3314.jpg: missing content file
  → skipping bair_stylized_feathers.jpg: missing content file
  → skipping bair_stylized_rain_princess.jpg

/opt/anaconda-2024.02/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda-2024.02/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
stats_ssim_from_folder("../StyTR-2/out_selftrained", "StyTr2_retrain", model="StyTr2")